In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
import numpy as np
import os
import shutil

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [10]:
def quasar_z_range(redshifts, psfmag, plate, mjd, fiberid): # this gives us the values of quasars we want
    
    fits_image_totalQ = fits.open("/Users/matt/Desktop/DESI_Research/DESI_ML/DR14_Q_spectra/DR14Q_v4_4.fits")
    hdul = fits_image_totalQ # this block of code is just reading in the data
    quasar_data = hdul["DR14Q_v4_4"].data
    
    redshift = quasar_data.field(str(redshifts)) # making the redshift vector
    psfmagtable = quasar_data.field(str(str(psfmag))) # the light magnitudes are in a table
    rmagvec = psfmagtable[:,2] # the r band is the 3 entry in the table, starts at 0 index
    plate = quasar_data.field(str(plate)) # making the plate vector
    mjd = quasar_data.field(str(mjd)) # making the mjd vector
    fiber = quasar_data.field(str(fiberid)) # making fiber vector
    
    cut_redshifts = (redshift >= 2.) & (redshift <= 3.) # gives boolean list with T in the range 2-3
    cut_rband = rmagvec < 19.5 # the range we're looking for in r band
    # ^ this range is opposite than what Professor Rockosi said, but it's the only way to get a sizable sample size
    cutlist = (cut_rband) & (cut_redshifts) # ma
    
    filtered_plate = np.array(plate)[np.array(cutlist)] # gives the values in the ID array for the Ts in cut list
    filtered_mjd = np.array(mjd)[np.array(cutlist)] # same as above but for the redshift array
    filtered_fiberid = np.array(fiber)[np.array(cutlist)] # same as above but for the noise array
    
    cut_values = {"PLATE":filtered_plate, "MJD":filtered_mjd, "FIBER":filtered_fiberid} # puts these new lists
                                                                                             # into a dictionary
    
    return cut_values


def in_range_quasars(redshifts, psfmag, plate, mjd, fiberid): # this gives the indice values of the quasars we want
    
    fits_image_totalQ = fits.open("/Users/matt/Desktop/DESI_Research/DESI_ML/DR14_Q_spectra/DR14Q_v4_4.fits")
    hdul = fits_image_totalQ # this block of code is just reading in the data
    quasar_data = hdul["DR14Q_v4_4"].data
    
    redshift = quasar_data.field(str(redshifts)) # making the redshift vector
    psfmagtable = quasar_data.field(str(str(psfmag))) # the light magnitudes are in a table
    rmagvec = psfmagtable[:,2] # the r band is the 3 entry in the table, starts at 0 index
    
    plate = quasar_data.field(str(plate)) # making the plate vector
    mjd = quasar_data.field(str(mjd)) # making the mjd vector
    fiber = quasar_data.field(str(fiberid)) # making fiber vector
    
    cut_redshifts = (redshift >= 1.8) & (redshift <= 2.2) # interval range we are looking at in redshift
    cut_rband = rmagvec <= 17. # the range we're looking for in r band
    # ^ this range is opposite than what Professor Rockosi said, but it's the only way to get a sizable sample size
    cutlist = (cut_rband) & (cut_redshifts) # making a boolean statement saying the quasar must be in the z & r range
    
    cut_plate = (cutlist) # making the boolean plate array with Ts & Fs
    cut_mjd = (cutlist) # "" mjd array ""
    cut_fiber = (cutlist) # "" fiber array ""
    
    true_plate = np.where(cut_plate)[0] # gives me the index values all the Ts in the plate array len ~ 80k quasars
    true_mjd = np.where(cut_mjd)[0] # "" mjd array
    true_fiber = np.where(cut_fiber)[0] # "" fiber array

    
    dict_values = {"PLATE":true_plate, "MJD":true_mjd, "FIBER":true_fiber} #, 'RBAND':cut_rband, "REDSHIFT":cut_redshifts}
    # ^ creates a dictionary for all the values I want to access
    
    return dict_values


def writeresultdict(resultsdict, outfileroot, outfileprefix=None):
    
    mainoutfilename = outfileroot +'all_quasars.txt'
    with open(mainoutfilename, 'w') as fout:
        donutsum = resultsdict#['donut_summary'] 
        for i in range(len(resultsdict['FIBER'])):
            outstr =  '{:04d}'.format((donutsum["PLATE"][i])) + '/spec'+'-' +'{:04d}'.format((donutsum["PLATE"][i]))+ '-'+\
            '{:05d}'.format(donutsum["MJD"][i]) + '-' + '{:04d}'.format(donutsum["FIBER"][i]) +  '.fits' + '\n'
            fout.write(outstr)

In [11]:
# intervalquasars = in_range_quasars("Z", "PSFMAG", "PLATE", "MJD", "FIBERID")
# writeresultdict(intervalquasars,"/Users/matt/Desktop/DESI_Research/DESI_ML/")

quasar_values = quasar_z_range("Z", "PSFMAG", "PLATE", "MJD", "FIBERID")

print("Min plate value = " +str(min(quasar_values['PLATE'])))
print("Max plate value = " +str(max(quasar_values['PLATE'])))
print('Number of good quasars = ' +str(len(quasar_values['PLATE'])))


Min plate value = 267
Max plate value = 10000
Number of good quasars = 21761


In [12]:
print(quasar_values['PLATE'][2])

7850


In [5]:
writeresultdict(quasar_values,"/Users/matt/Desktop/DESI_Research/DESI_ML/")

In [6]:
fits_image_totalQ = fits.open("/Users/matt/Desktop/DESI_Research/DESI_ML/DR14_Q_spectra/DR14Q_v4_4.fits")
hdul = fits_image_totalQ # this block of code is just reading in the data
quasar_data = hdul["DR14Q_v4_4"].data

In [7]:
Table(quasar_data)

SDSS_NAME,RA,DEC,THING_ID,PLATE,MJD,FIBERID,SPECTRO,Z,Z_ERR,SOURCE_Z,Z_VI,Z_PIPE,Z_PIPE_ERR,ZWARNING,Z_PCA,Z_PCA_ER,Z_MGII,BOSS_TARGET1,ANCILLARY_TARGET1,ANCILLARY_TARGET2,EBOSS_TARGET0,EBOSS_TARGET1,EBOSS_TARGET2,N_SPEC_SDSS,N_SPEC_BOSS,N_SPEC,PLATE_DUPLICATE [51],MJD_DUPLICATE [51],FIBERID_DUPLICATE [51],SPECTRO_DUPLICATE [51],BI_CIV,ERR_BI_CIV,RUN_NUMBER,RERUN_NUMBER,COL_NUMBER,FIELD_NUMBER,OBJ_ID,PSFFLUX [5],IVAR_PSFFLUX [5],PSFMAG [5],ERR_PSFMAG [5],MI,GAL_EXT [5],RASS_COUNTS,RASS_COUNTS_SNR,SDSS2ROSAT_SEP,FLUX_0.2_2.0keV,FLUX_0.2_2.0keV_ERR,FLUX_2.0_12.0keV,FLUX_2.0_12.0keV_ERR,FLUX_0.2_12.0keV,FLUX_0.2_12.0keV_ERR,LUM_0.2_12.0keV,SDSS2XMM_SEP,GALEX_MATCHED,FUV,FUV_IVAR,NUV,NUV_IVAR,JMAG,ERR_JMAG,JSNR,JRDFLAG,HMAG,ERR_HMAG,HSNR,HRDFLAG,KMAG,ERR_KMAG,KSNR,KRDFLAG,SDSS2MASS_SEP,W1MAG,ERR_W1MAG,W1SNR,W1CHI2,W2MAG,ERR_W2MAG,W2SNR,W2CHI2,W3MAG,ERR_W3MAG,W3SNR,W3CHI2,W4MAG,ERR_W4MAG,W4SNR,W4CHI2,CC_FLAGS,PH_FLAGS,SDSS2WISE_SEP,UKIDSS_MATCHED,YFLUX,YFLUX_ERR,JFLUX,JFLUX_ERR,HFLUX,HFLUX_ERR,KFLUX,KFLUX_ERR,FIRST_MATCHED,FIRST_FLUX,FIRST_SNR,SDSS2FIRST_SEP
str18,float64,float64,int32,int16,int32,int16,str4,float64,float64,str4,float64,float64,float64,int16,float64,float64,float64,int64,int64,int64,int64,int64,int64,int16,int16,int16,int32,int32,int32,str1,float64,float64,int16,str1,int16,int16,str1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float64,float64,float64,float64,float64,float64,float32,int16,float64,float64,float32,int16,float64,float64,float32,int16,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str4,str4,float64,float32,float64,float64,float64,float64,float64,float64,float64,float64,int16,float64,float64,float64
000000.45+174625.4,0.001898274356335605,17.773739123771385,268270422,7596,56945,162,BOSS,2.309,0.0,VI,2.309000015258789,2.307750701904297,0.0010300044668838382,0,2.3045657000000004,-1.0,-1.0,0,0,0,0,4096,0,0,1,1,0 .. -1,0 .. -1,0 .. -1,- .. -,0.0,0.0,0,-,0,0,-,1.0379366874694824 .. 2.631136417388916,12.372532844543457 .. 0.8706143498420715,22.440338134765625 .. 21.372461318969727,0.2871243953704834 .. 0.38545534014701843,-24.410782,0.12021728605031967 .. 0.03581845760345459,0.0,0.0,-1.0,nan,nan,nan,nan,nan,nan,nan,nan,1.0,0.30231887102127075,8.785573140381722,0.7275648713111877,4.537358964921044,nan,nan,nan,-1,nan,nan,nan,-1,nan,nan,nan,-1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,,,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1.0,-1.0,-1.0
000000.57+055630.8,0.0023754761888312714,5.941903005523435,163544156,8740,57367,694,BOSS,2.1031202999999996,0.0,AUTO,-1.0,2.1031203269958496,0.0009241577354259787,0,2.0996384999999997,2.0983490000000002,2.0983490000000002,0,0,0,0,2048,0,0,0,0,-1 .. -1,-1 .. -1,-1 .. -1,- .. -,0.0,0.0,0,-,0,0,-,1.1197296380996704 .. 4.730101585388184,9.196866035461426 .. 0.8192309737205505,22.360628128051758 .. 20.78717613220215,0.3101848363876343 .. 0.2420300394296646,-24.732,0.2338263988494873 .. 0.06966802477836609,0.0,0.0,-1.0,nan,nan,nan,nan,nan,nan,nan,nan,1.0,0.0,0.0,0.0,0.0,nan,nan,nan,-1,nan,nan,nan,-1,nan,nan,nan,-1,nan,16.969,0.127,8.5,0.9904,16.887999999999998,0.409,2.700000047683716,1.004,11.861,nan,1.399999976158142,0.9819,8.604,0.127,0.699999988079071,0.9478,hH00,BCUU,0.706171,1.0,1.2885136657753169e-31,2.4790984841010035e-32,1.229889058862803e-31,4.504542253745802e-32,1.496424644649041e-31,4.7290496259357616e-32,2.4598190249290145e-31,5.689571920868372e-32,0,-1.0,-1.0,-1.0
000000.66+145828.8,0.002756432550313548,14.974675485824303,244864389,6177,56268,595,BOSS,2.497,0.0,VI,2.496999979019165,2.49796199798584,0.0008690897957421839,0,2.5099661,-1.0,-1.0,2199023255552,0,0,0,0,0,0,1,1,0 .. -1,0 .. -1,0 .. -1,- .. -,0.0,0.0,0,-,0,0,-,1.660184383392334 .. 5.348500728607178,15.485949516296387 .. 0.861799418926239,21.94196891784668 .. 20.659208297729492,0.16387343406677246 

In [14]:
# this code copies the fits in their respective folders into one folder

def copyfiles_fromfolder_tofolder(Root_dir,target_folder,extension):
    RootDir1 = str(Root_dir)
    TargetFolder = str(target_folder)
    for root, dirs, files in os.walk((os.path.normpath(RootDir1)), topdown=False):
        for name in files:
            if name.endswith(str(extension)):
                SourceFolder = os.path.join(root,name)
                shutil.copy2(SourceFolder, TargetFolder)


# this is the function code w/ example

# RootDir1 = "/Users/matt/Desktop/DESI_Research/DESI_ML/Valid_Quasar_Data"
# TargetFolder = "/Users/matt/Desktop/DESI_Research/DESI_ML/good_quasars"
# for root, dirs, files in os.walk((os.path.normpath(RootDir1)), topdown=False):
#         for name in files:
#             if name.endswith('.fits'):
#                 #print("Found")
#                 SourceFolder = os.path.join(root,name)
#                 shutil.copy2(SourceFolder, TargetFolder) #copies csv to new folder



In [9]:
RootDir1 = "/Users/matt/Desktop/DESI_Research/DESI_ML/Valid_all_quasars"
TargetFolder = "/Users/matt/Desktop/DESI_Research/DESI_ML/good_all_quasars"
for root, dirs, files in os.walk((os.path.normpath(RootDir1)), topdown=False):
        for name in files:
            if name.endswith('.fits'):
                #print("Found")
                SourceFolder = os.path.join(root,name)
                shutil.copy2(SourceFolder, TargetFolder)

In [ ]:
RootDir1 = "/Users/matt/Desktop/DESI_Research/DESI_ML/Valid_all_stars"
TargetFolder = "/Users/matt/Desktop/DESI_Research/DESI_ML/good_all_stars"
for root, dirs, files in os.walk((os.path.normpath(RootDir1)), topdown=False):
        for name in files:
            if name.endswith('.fits'):
                #print("Found")
                SourceFolder = os.path.join(root,name)
                shutil.copy2(SourceFolder, TargetFolder)